# Data Info

The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images that are commonly used to train machine learning and computer vision algorithms.

The detailed information can be found at https://github.com/Priyanshuuu/Project-Cifar10

# Library Import

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import ParameterGrid


# Load Data

In [3]:
# Load and preprocess the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 170498071/170498071 [00:06<00:00, 25499695.06it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Define a CNN architecture
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # Fully Connected Layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# Instantiate the model, define the loss function and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Training function
def train(model, criterion, optimizer, loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(loader, 0):
            inputs, labels = data

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

In [7]:
# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on the test images: {accuracy:.2f}%')


In [8]:
# Train the model
train(model, criterion, optimizer, train_loader, num_epochs=10)

[Epoch 1, Batch 100] Loss: 1.9868
[Epoch 1, Batch 200] Loss: 1.5761
[Epoch 1, Batch 300] Loss: 1.4316
[Epoch 1, Batch 400] Loss: 1.3573
[Epoch 1, Batch 500] Loss: 1.2533
[Epoch 1, Batch 600] Loss: 1.1944
[Epoch 1, Batch 700] Loss: 1.1588
[Epoch 2, Batch 100] Loss: 1.0244
[Epoch 2, Batch 200] Loss: 0.9914
[Epoch 2, Batch 300] Loss: 0.9616
[Epoch 2, Batch 400] Loss: 0.9216
[Epoch 2, Batch 500] Loss: 0.8934
[Epoch 2, Batch 600] Loss: 0.8560
[Epoch 2, Batch 700] Loss: 0.8447
[Epoch 3, Batch 100] Loss: 0.7800
[Epoch 3, Batch 200] Loss: 0.7444
[Epoch 3, Batch 300] Loss: 0.7345
[Epoch 3, Batch 400] Loss: 0.7234
[Epoch 3, Batch 500] Loss: 0.7221
[Epoch 3, Batch 600] Loss: 0.7090
[Epoch 3, Batch 700] Loss: 0.7269
[Epoch 4, Batch 100] Loss: 0.5900
[Epoch 4, Batch 200] Loss: 0.5926
[Epoch 4, Batch 300] Loss: 0.5929
[Epoch 4, Batch 400] Loss: 0.5881
[Epoch 4, Batch 500] Loss: 0.5836
[Epoch 4, Batch 600] Loss: 0.5866
[Epoch 4, Batch 700] Loss: 0.5895
[Epoch 5, Batch 100] Loss: 0.4684
[Epoch 5, Batc

In [9]:
# Evaluate the model
evaluate(model, test_loader)

Accuracy on the test images: 75.24%


# Tune model

In [10]:
# Load and preprocess the CIFAR-10 dataset with data augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomCrop(32, padding=4),  # Crop with padding
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define an improved CNN architecture
class EnhancedCNN(nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # Dropout Layers
        self.dropout = nn.Dropout(0.4)
        # Fully Connected Layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Instantiate the model, define the loss function and optimizer
model = EnhancedCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Training function with model checkpointing
def train(model, criterion, optimizer, loader, num_epochs=10):
    model.train()
    best_accuracy = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(loader, 0):
            inputs, labels = data

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

        # Evaluate the model at the end of each epoch
        accuracy = evaluate(model, test_loader, verbose=False)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), "best_model.pth")
            print(f'Best model saved with accuracy: {accuracy:.2f}%')

# Evaluation function
def evaluate(model, loader, verbose=True):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    if verbose:
        print(f'Accuracy on the test images: {accuracy:.2f}%')
    return accuracy

# Train the model
train(model, criterion, optimizer, train_loader, num_epochs=10)

# Load and evaluate the best model
best_model = EnhancedCNN()
best_model.load_state_dict(torch.load("best_model.pth"))
evaluate(best_model, test_loader)


Files already downloaded and verified
Files already downloaded and verified
[Epoch 1, Batch 100] Loss: 2.1479
[Epoch 1, Batch 200] Loss: 1.8723
[Epoch 1, Batch 300] Loss: 1.7453
[Epoch 1, Batch 400] Loss: 1.6380
[Epoch 1, Batch 500] Loss: 1.5850
[Epoch 1, Batch 600] Loss: 1.5218
[Epoch 1, Batch 700] Loss: 1.4557
Best model saved with accuracy: 52.27%
[Epoch 2, Batch 100] Loss: 1.3305
[Epoch 2, Batch 200] Loss: 1.3118
[Epoch 2, Batch 300] Loss: 1.2477
[Epoch 2, Batch 400] Loss: 1.2064
[Epoch 2, Batch 500] Loss: 1.2024
[Epoch 2, Batch 600] Loss: 1.1732
[Epoch 2, Batch 700] Loss: 1.1607
Best model saved with accuracy: 64.18%
[Epoch 3, Batch 100] Loss: 1.0629
[Epoch 3, Batch 200] Loss: 1.0621
[Epoch 3, Batch 300] Loss: 1.0588
[Epoch 3, Batch 400] Loss: 1.0133
[Epoch 3, Batch 500] Loss: 0.9851
[Epoch 3, Batch 600] Loss: 0.9682
[Epoch 3, Batch 700] Loss: 0.9933
Best model saved with accuracy: 65.83%
[Epoch 4, Batch 100] Loss: 0.9355
[Epoch 4, Batch 200] Loss: 0.8998
[Epoch 4, Batch 300] Loss

78.25

In [12]:

# Define hyperparameters to tune
param_grid = {
    'lr': [0.001, 0.0005],
    'weight_decay': [1e-4, 1e-5],
    'dropout_rate': [0.4, 0.5]
}

# Define the grid search function
def hyperparameter_search(param_grid):
    best_accuracy = 0
    best_params = None

    for params in ParameterGrid(param_grid):
        print(f'Testing parameters: {params}')
        # Define a new model architecture using the current parameters
        class GridSearchCNN(nn.Module):
            def __init__(self):
                super(GridSearchCNN, self).__init__()
                self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
                self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
                self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
                self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                self.dropout = nn.Dropout(params['dropout_rate'])
                self.fc1 = nn.Linear(128 * 4 * 4, 512)
                self.fc2 = nn.Linear(512, 256)
                self.fc3 = nn.Linear(256, 10)

            def forward(self, x):
                x = self.pool(F.relu(self.conv1(x)))
                x = self.pool(F.relu(self.conv2(x)))
                x = self.pool(F.relu(self.conv3(x)))
                x = x.view(-1, 128 * 4 * 4)
                x = F.relu(self.fc1(x))
                x = self.dropout(x)
                x = F.relu(self.fc2(x))
                x = self.dropout(x)
                x = self.fc3(x)
                return x

        # Instantiate the model with the new parameters
        model = GridSearchCNN()
        optimizer = optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
        criterion = nn.CrossEntropyLoss()

        # Train the model (use fewer epochs for grid search)
        train(model, criterion, optimizer, train_loader, num_epochs=5)
        
        # Evaluate on the test set
        accuracy = evaluate(model, test_loader, verbose=False)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params

    print(f'Best parameters found: {best_params} with accuracy: {best_accuracy:.2f}%')
    return best_params

# Perform hyperparameter tuning
best_params = hyperparameter_search(param_grid)

# Use best_params to retrain the final model
final_model = EnhancedCNN()
optimizer = optim.Adam(final_model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])
criterion = nn.CrossEntropyLoss()
train(final_model, criterion, optimizer, train_loader, num_epochs=10)
evaluate(final_model, test_loader)


Testing parameters: {'dropout_rate': 0.4, 'lr': 0.001, 'weight_decay': 0.0001}
[Epoch 1, Batch 100] Loss: 2.1225
[Epoch 1, Batch 200] Loss: 1.8913
[Epoch 1, Batch 300] Loss: 1.7626
[Epoch 1, Batch 400] Loss: 1.6762
[Epoch 1, Batch 500] Loss: 1.5939
[Epoch 1, Batch 600] Loss: 1.5471
[Epoch 1, Batch 700] Loss: 1.4791
Best model saved with accuracy: 53.25%
[Epoch 2, Batch 100] Loss: 1.3605
[Epoch 2, Batch 200] Loss: 1.2900
[Epoch 2, Batch 300] Loss: 1.2822
[Epoch 2, Batch 400] Loss: 1.2469
[Epoch 2, Batch 500] Loss: 1.2057
[Epoch 2, Batch 600] Loss: 1.1729
[Epoch 2, Batch 700] Loss: 1.1417
Best model saved with accuracy: 62.02%
[Epoch 3, Batch 100] Loss: 1.0937
[Epoch 3, Batch 200] Loss: 1.0894
[Epoch 3, Batch 300] Loss: 1.0440
[Epoch 3, Batch 400] Loss: 1.0152
[Epoch 3, Batch 500] Loss: 1.0295
[Epoch 3, Batch 600] Loss: 1.0035
[Epoch 3, Batch 700] Loss: 1.0437
Best model saved with accuracy: 67.14%
[Epoch 4, Batch 100] Loss: 0.9330
[Epoch 4, Batch 200] Loss: 0.9838
[Epoch 4, Batch 300] L

78.29